In [95]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from numpy.polynomial.chebyshev import Chebyshev
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.nn as nn
import torch.optim as optim
from pde import CartesianGrid, ScalarField, solve_poisson_equation
import copy 
import pandas as pd
device = torch.device("cpu")

## 1 - Construct the Dataset 

In [70]:
def sample_function(class_type, x_grid, coeff_range = (-3, 3), max_n_polynomals = 10):
    if class_type == "GP":
        kernel = RBF(length_scale=1.0)
        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=10, normalize_y=True)
        f = gp.sample_y(x_grid.reshape(-1, 1), 1).flatten()
    elif class_type == "PL":
        num_pieces = np.random.randint(
            2, 10
        ) 
        jumps = np.sort(np.random.choice(x_grid, num_pieces, replace=False))
        slopes = np.random.uniform(*coeff_range, num_pieces - 1)
        intercepts = np.random.uniform(*coeff_range, num_pieces - 1)

        f = np.zeros_like(x_grid)
        for i in range(1, len(jumps)):
            mask = (x_grid >= jumps[i - 1]) & (x_grid <= jumps[i])
            f[mask] = slopes[i - 1] * x_grid[mask] + intercepts[i - 1]
    elif class_type == "CP":
        # Chebyshev Polynomials
        num_polynomials = np.random.randint(1, max_n_polynomals)
        coeffs =  np.random.uniform(*coeff_range, num_polynomials)
        cheb_poly = Chebyshev(coeffs, domain=[-1, 1])
        f = cheb_poly(x)

    f[0] = 0
    f[-1] = 0
    
    return f

In [71]:
def solve_poisson(f, N):
    """
    Solve the Poisson equation with the given function f and grid size N

    Args:
    f: function to solve for
    N: grid size

    Returns:
    result: solution to the Poisson equation
    """
    # Define the grid
    grid = CartesianGrid([[-1, 1]], N)
    # Defined the scalar field
    field = ScalarField(grid, data=f)
    # Define the PDE problem with the boundary conditions
    result =  solve_poisson_equation(field, bc=({"value": 0}, {"value": 0}))

    return result.data

In [72]:
# Initialize dataset
N = 100
x = np.linspace(-1, 1, N)

D = {"GP": [], "PL": [], "CP": []}

for classe in ["GP", "PL", "CP"]:
    for i in range(100):
        f = sample_function(classe, x)
        u = solve_poisson(f, N)
        D[classe].append((f, x, u))

In [73]:
np.array(D["GP"]).shape

(100, 3, 100)

In [74]:
def create_loaders(D, batch_size=16, split_ratio=0.8):

    loaders  = {"GP": [], "PL": [], "CP": []}

    for classe in ["GP", "PL", "CP"]:
        data = np.array(D[classe])
        inputs = torch.tensor(data[:, :2, :], dtype=torch.float32).to(device).permute(0, 2, 1)
        outputs = torch.tensor(data[:, 2, :], dtype=torch.float32).to(device).unsqueeze(1)
        torch_dataset = TensorDataset(inputs, outputs)
        train_set, test_set = torch.utils.data.random_split(torch_dataset, [int(len(torch_dataset) * split_ratio), len(torch_dataset) - int(len(torch_dataset) * split_ratio)])
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
        loaders[classe] = (train_loader, test_loader)
    
    return loaders

loaders = create_loaders(D, batch_size=32, split_ratio=0.8)

## 2 - Train Neural Operators

In [75]:
# Use FNO1d class from tutorial 

class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1

        self.scale = 1 / (in_channels * out_channels)
        self.weights1 = nn.Parameter(
            self.scale
            * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat)
        )

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # x.shape == [batch_size, in_channels, number of grid points]
        # hint: use torch.fft library torch.fft.rfft
        # use DFT to approximate the fourier transform

        # Compute Fourier coefficients
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(
            batchsize,
            self.out_channels,
            x.size(-1) // 2 + 1,
            device=x.device,
            dtype=torch.cfloat,
        )
        out_ft[:, :, : self.modes1] = self.compl_mul1d(
            x_ft[:, :, : self.modes1], self.weights1
        )

        # Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x


class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .

        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.padding = 1  # pad the domain if input is non-periodic
        self.linear_p = nn.Linear(
            2, self.width
        )  # input channel is 2: (u0(x), x) --> GRID IS INCLUDED!

        self.spect1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.spect3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.lin0 = nn.Conv1d(self.width, self.width, 1)
        self.lin1 = nn.Conv1d(self.width, self.width, 1)
        self.lin2 = nn.Conv1d(self.width, self.width, 1)

        self.linear_q = nn.Linear(self.width, 32)
        self.output_layer = nn.Linear(32, 1)

        self.activation = torch.nn.Tanh()

    def fourier_layer(self, x, spectral_layer, conv_layer):
        return self.activation(spectral_layer(x) + conv_layer(x))

    def linear_layer(self, x, linear_transformation):
        return self.activation(linear_transformation(x))

    def forward(self, x):
        # grid = self.get_grid(x.shape, x.device)
        # x = torch.cat((x, grid), dim=-1)
        x = self.linear_p(x)
        x = x.permute(0, 2, 1)

        # x = F.pad(x, [0, self.padding])  # pad the domain if input is non-periodic

        x = self.fourier_layer(x, self.spect1, self.lin0)
        x = self.fourier_layer(x, self.spect2, self.lin1)
        x = self.fourier_layer(x, self.spect3, self.lin2)

        # x = x[..., :-self.padding]  # pad the domain if input is non-periodic
        x = x.permute(0, 2, 1)

        x = self.linear_layer(x, self.linear_q)
        x = self.output_layer(x)
        return x

In [78]:
neural_operators = {"GP": FNO1d(16, 64), "PL": FNO1d(16, 64), "CP": FNO1d(16, 64)}

l_r = 0.001
epochs = 100
batch_size = 32

for classe in ["GP", "PL", "CP"]:
    optimizer = optim.Adam(neural_operators[classe].parameters(), lr=l_r)
    criterion = nn.MSELoss()
    train_loader = loaders[classe][0]
    neural_operators[classe].to(device)
    neural_operators[classe].train()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = neural_operators[classe](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, loss: {running_loss / len(train_loader)}")


Epoch 0, loss: 0.12207488715648651
Epoch 10, loss: 0.055168223877747856
Epoch 20, loss: 0.05417410532633463
Epoch 30, loss: 0.0541255809366703
Epoch 40, loss: 0.05412499109903971
Epoch 50, loss: 0.05412482718626658
Epoch 60, loss: 0.05412473653753599
Epoch 70, loss: 0.05412467444936434
Epoch 80, loss: 0.05412463843822479
Epoch 90, loss: 0.05412461732824644
Epoch 0, loss: 0.10641288260618846
Epoch 10, loss: 0.03483750236531099
Epoch 20, loss: 0.03074645499388377
Epoch 30, loss: 0.03242842108011246
Epoch 40, loss: 0.03232711801926295
Epoch 50, loss: 0.03307214441398779
Epoch 60, loss: 0.02976777528723081
Epoch 70, loss: 0.03241293070216974
Epoch 80, loss: 0.03163355775177479
Epoch 90, loss: 0.029726338262359302
Epoch 0, loss: 0.5194552143414816
Epoch 10, loss: 0.13301007449626923
Epoch 20, loss: 0.11464632550875346
Epoch 30, loss: 0.10252510259548824
Epoch 40, loss: 0.10898800194263458
Epoch 50, loss: 0.10055247445901234
Epoch 60, loss: 0.10451279083887736
Epoch 70, loss: 0.1052966068188

## 3 - Evaluation: Zero-Shot and Finetuning

In [105]:
# create the array where to store the results: as many columns as possible combinations of
# classes and three columns for the zero-shot loss, the finetuned loss and the improvement
R = np.array([])
epochs_finetuing = 30

for i, classe1 in enumerate(["GP", "PL", "CP"]):
    for j, classe2 in enumerate(["GP", "PL", "CP"]):
        print(f"Comparing model from {classe1} and evaluating on {classe2}")
        # do deep copy of the neural operator to avoid overwriting
        model = copy.deepcopy(neural_operators[classe1])

        ## Zero-shot loss ##
        test_loader = loaders[classe2][1]

        criterion = nn.MSELoss()
        model.eval()
        zero_shot_loss = 0
        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                inputs, labels = data
                outputs = model(inputs)
                loss = criterion(outputs, labels) 
                zero_shot_loss += loss.item()
        zero_shot_loss /= len(test_loader)

        # Finetuned loss
        optimizer = optim.Adam(model.parameters(), lr=l_r)
        model.to(device)
        model.train()
        # get 20 random samples from the test loader
        test_loader_finetune = DataLoader(
            Subset(test_loader.dataset, np.random.choice(len(test_loader.dataset), 20)),
            batch_size=20,
        )

        for epoch in range(epochs_finetuing):
            running_loss = 0.0
            for i, data in enumerate(test_loader_finetune, 0):
                inputs, labels = data
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            if epoch % 5 == 0:
                print(f"Epoch {epoch}, loss: {running_loss / len(test_loader_finetune)}")

        model.eval()
        finetuned_loss = 0
        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                inputs, labels = data
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                finetuned_loss += loss.item()
        finetuned_loss /= len(test_loader)

        # stack the results in the array
        R = np.append(R, np.array([classe1, classe2, round(zero_shot_loss, 3), round(finetuned_loss, 3), round(100 * (zero_shot_loss - finetuned_loss) /zero_shot_loss, 3)]))

        print("\n")

columns = [
    "Model trained on",
    "Model evaluated on",
    "Zero-shot loss",
    "Finetuned loss",
    "relative L2 error improvement (%)",
]
df = pd.DataFrame(R.reshape(9,5), columns=columns)
blankIndex = [""] * len(df)
df.index = blankIndex
df

Comparing model from GP and evaluating on GP
Epoch 0, loss: 0.054124604910612106


/Users/lorenzotarricone/miniconda3/envs/torch_tutorial/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1, 100])) that is different to the input size (torch.Size([20, 100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 5, loss: 0.05575158819556236
Epoch 10, loss: 0.054807908833026886
Epoch 15, loss: 0.0542176216840744
Epoch 20, loss: 0.05415672808885574
Epoch 25, loss: 0.054184917360544205


Comparing model from GP and evaluating on PL
Epoch 0, loss: 0.16372349858283997
Epoch 5, loss: 0.05774083361029625
Epoch 10, loss: 0.051015228033065796
Epoch 15, loss: 0.052244994789361954
Epoch 20, loss: 0.04645328223705292
Epoch 25, loss: 0.046484436839818954


Comparing model from GP and evaluating on CP
Epoch 0, loss: 0.34076589345932007
Epoch 5, loss: 0.09462401270866394
Epoch 10, loss: 0.08999630063772202
Epoch 15, loss: 0.09238732606172562
Epoch 20, loss: 0.08736167103052139
Epoch 25, loss: 0.08164644241333008


Comparing model from PL and evaluating on GP
Epoch 0, loss: 0.061840325593948364
Epoch 5, loss: 0.07124020904302597
Epoch 10, loss: 0.063709557056427
Epoch 15, loss: 0.05962558090686798
Epoch 20, loss: 0.05594318360090256
Epoch 25, loss: 0.05414467304944992


Comparing model from PL and evalu

,Model trained on,Model evaluated on,Zero-shot loss,Finetuned loss,relative L2 error improvement (%)
,GP,GP,0.054,0.054,-0.154
,GP,PL,0.14,0.035,75.386
,GP,CP,0.369,0.093,74.673
,PL,GP,0.062,0.054,12.034
,PL,PL,0.032,0.032,0.576
,PL,CP,0.082,0.082,0.138
,CP,GP,0.054,0.055,-1.757
,CP,PL,0.037,0.032,13.929
,CP,CP,0.075,0.078,-3.465


In [106]:
# Save the results
df.to_csv("results.csv", index=False)

In [107]:
#to insert in the report 
print(df.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
Model trained on & Model evaluated on & Zero-shot loss & Finetuned loss & relative L2 error improvement (%) \\
\midrule
GP & GP & 0.054 & 0.054 & -0.154 \\
GP & PL & 0.14 & 0.035 & 75.386 \\
GP & CP & 0.369 & 0.093 & 74.673 \\
PL & GP & 0.062 & 0.054 & 12.034 \\
PL & PL & 0.032 & 0.032 & 0.576 \\
PL & CP & 0.082 & 0.082 & 0.138 \\
CP & GP & 0.054 & 0.055 & -1.757 \\
CP & PL & 0.037 & 0.032 & 13.929 \\
CP & CP & 0.075 & 0.078 & -3.465 \\
\bottomrule
\end{tabular}

